# Getting Started with Core4AI in Jupyter Notebooks

Core4AI (Contextual Optimization and Refinement Engine for AI) is a powerful tool that transforms basic user queries into optimized prompts for AI systems using MLflow Prompt Registry. In this guide, we'll explore how to set up and use Core4AI in a Jupyter Notebook environment, covering everything from initial configuration to practical applications.

## Table of Contents

1. [Environment Setup](#environment-setup)
2. [Initial Configuration](#initial-configuration)
3. [Working with Prompt Templates](#working-with-prompt-templates)
4. [Switching Between AI Providers](#switching-between-ai-providers)
5. [Generating Enhanced Responses](#generating-enhanced-responses)
6. [Summary](#summary)

## Environment Setup

First, let's set up our environment by installing Core4AI and its dependencies. In a notebook cell, run:

In [ ]:
# Install Core4AI
!pip install core4ai==1.1.5

In [ ]:
# Import required libraries
import os
import json
from pathlib import Path
from core4ai import Core4AI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

## Initial Configuration

### Connecting to MLflow Server

Core4AI uses MLflow to store and manage prompt templates. Let's configure it to connect to a local MLflow server running on port 8080:

In [2]:
# Create a Core4AI instance
ai = Core4AI()

In [ ]:
#configure MLflow
ai.set_mlflow_uri("http://localhost:8080")

# Verify configuration
ai.get_current_config()

### Setting Up OpenAI Provider

To use OpenAI as the default provider:

In [ ]:
# Set up OpenAI (using environment variable for API key)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
ai.configure_openai(model="gpt-3.5-turbo")

# Save configuration
ai.save_config()

### Setting Up Ollama Provider

Alternatively, if you're using Ollama (local LLM server):

```python
# Set up Ollama
ai.configure_ollama(uri="http://localhost:11434", model="llama3.2:latest")

# Save configuration
ai.save_config()
```

## Working with Prompt Templates

### Registering Sample Prompts

Core4AI comes with pre-defined prompt templates. Let's register them:

In [ ]:
# Register sample prompts
result = ai.register_samples()
print(f"Registered {result.get('registered', 0)} prompts")

### Listing Available Prompts

To see what prompts are available:

In [ ]:
# List all available prompts
prompts = ai.list_prompts()

# Display prompt names and types
if prompts.get("status") == "success":
    prompt_list = prompts.get("prompts", [])
    for p in prompt_list:
        print(f"{p['name']} ({p['type']}) - Variables: {', '.join(p['variables'])}")

### Creating Custom Prompt Templates

You can create your own prompt templates:

In [ ]:
# Create a new prompt template
template_result = ai.create_prompt_template("data_visualization_prompt", 
                                           output_dir="./my_prompts")

# The template file is created at the specified location
# Edit it manually, then register it
if template_result.get("status") == "success":
    file_path = template_result.get("file_path")
    print(f"Edit the template at: {file_path} and then register it")
    
    # After editing, register the template
    ai.register_from_markdown(file_path)

### Direct Registration

You can also register prompts directly:

In [ ]:
# Register a prompt directly
ai.register_prompt(
    name="quick_analysis_prompt",
    template="Analyze the following {{ data_type }} data and provide {{ analysis_type }} insights:\n\n```\n{{ data }}\n```\n\nFocus on {{ focus_area }} in your analysis.",
    tags={"type": "quick_analysis", "task": "data_analysis"}
)

## Switching Between AI Providers

Core4AI allows easy switching between providers:

In [ ]:
# Switch to OpenAI
def use_openai():
    ai.configure_openai(model="gpt-4o")
    ai.save_config()
    print("Now using OpenAI with model: gpt-4o")
    
# Switch to Ollama
def use_ollama():
    ai.configure_ollama(uri="http://localhost:11434", model="llama3.2:latest")
    ai.save_config()
    print("Now using Ollama with model: llama3.2:latest")

# Use OpenAI for this example
use_openai()

## Generating Enhanced Responses

Now let's use Core4AI to generate enhanced responses:

In [ ]:
# Simple query
response = ai.chat("Write about the benefits of machine learning")
print(response["response"])


### Getting Enhanced Query Details

To see how Core4AI enhances queries:

In [ ]:
# Verbose output to see the enhancement process
response = ai.chat("Write an email to my boss about a project delay", verbose=True)

print("Original Query:", response["original_query"])
print("\nMatched to:", response["prompt_match"].get("prompt_name", "No match"))
print("Confidence:", response["prompt_match"].get("confidence", 0), "%")
print("\nEnhanced Query:", response["enhanced_query"])
print("\nResponse:", response["response"])

### Working with Specific Prompt Types

To use a specific prompt type:

In [ ]:
# Craft a query that clearly matches a technical prompt
response = ai.chat("Explain how neural networks work to a beginner", verbose=True)
print(f"Matched to: {response['prompt_match'].get('prompt_name')}")
print(f"Content type: {response['content_type']}")
print("\nResponse:", response["response"])

### Content Type Detection

Core4AI automatically detects content types:

In [ ]:
# Test various query types to see content detection
test_queries = [
    "Write an essay about climate change",
    "Draft an email to a client about project progress",
    "Create a Python function to sort a list",
    "Compare electric cars and gasoline cars",
    "Summarize the key points about quantum computing"
]

for query in test_queries:
    response = ai.chat(query)
    content_type = response.get("content_type", "Unknown")
    print(f"Query: '{query}'")
    print(f"Detected content type: {content_type}")
    print(f"Matched prompt: {response['prompt_match'].get('prompt_name', 'None')}\n")

## Summary

This guide has demonstrated how to:
- Configure Core4AI in a Jupyter Notebook
- Register and manage prompt templates
- Switch between AI providers
- Generate enhanced responses

Core4AI provides a powerful abstraction layer that helps you create more consistent, high-quality AI responses by automatically matching user queries to appropriate prompt templates and optimizing them before sending to the LLM provider.

The ability to switch seamlessly between cloud-based providers like OpenAI and local models via Ollama gives you flexibility to choose the right solution for your specific needs, whether you need top-tier capabilities or prefer local privacy-focused deployment.

Happy prompting!